# DecisionTree Assignment - 20기 조민영

물음표 친 부분을 채우고 코드에 대한 주석을 자세하게 달아주세요!

# Data Loading

In [183]:
import pandas as pd 
import numpy as np

In [2]:
pd_data = pd.read_csv('https://raw.githubusercontent.com/AugustLONG/ML01/master/01decisiontree/AllElectronics.csv')
pd_data.drop("RID",axis=1, inplace = True) #RID는 그냥 순서라서 삭제
pd_data

,age,income,student,credit_rating,class_buys_computer
0,youth,high,no,fair,no
1,youth,high,no,excellent,no
2,middle_aged,high,no,fair,yes
3,senior,medium,no,fair,yes
4,senior,low,yes,fair,yes
5,senior,low,yes,excellent,no
6,middle_aged,low,yes,excellent,yes
7,youth,medium,no,fair,no
8,youth,low,yes,fair,yes
9,senior,medium,yes,fair,yes


## Gini 계수를 구하는 함수 만들기

<img src="gini.png" width="200">

- Input: df(데이터), label(타겟변수명)
- 해당 결과는 아래와 같이 나와야 합니다.

- 지니계수는 데이터의 통계적 분산 정도를 정량화 해서 표현한 값이다.
- 어떤 집합의 gini index가 높을수록 그 집단의 데이터가 분산되어 있음을 확인할 수 있다.

In [34]:
def get_gini(df, label):
    
    # 타겟변수에 대해서만 데이터프레임 추출
    data = df[label]
    
    # gini의 초기값을 1로 설정한다.
    gini = 1
    for a in data.unique():  # 데이터프레임의 원소를 가져와서
        gini -= (sum(data==a)/len(data))**2   # 확률값의 제곱을 1에서 빼나간다.
        
    return gini

In [35]:
get_gini(pd_data,'class_buys_computer')

0.4591836734693877

## Feature의 Class를 이진 분류로 만들기
- ex) {A,B,C} -> ({A}, {B,C}), ({B}, {A,C}), ({C}, {A,B})

- Input: df(데이터), attribute(Gini index를 구하고자 하는 변수명)
- Income 변수를 결과로 출력해주세요.

In [5]:
from itertools import combinations

def get_binary_split(df, attribute):
    
    uniques = list(df[attribute].unique()) # 속성 데이터 고유값들을 담은 리스트 
    
    #1개씩 묶는 조합을 담은 리스트
    nC1 = list(combinations(uniques, 1))   # 결과 : [('high',), ('medium',), ('low',)]
    
    #2개씩 묶는 조합을 담은 리스트
    nC2 = list(combinations(uniques, 2))   # 결과 : [('high', 'medium'), ('high', 'low'), ('medium', 'low')]
    
    result = []
    for a in (nC1 + nC2) :      # 위에서 만든 두 개의 리스트를 합친 다음 원소를 가져온다.
        result.append(list(a))   # 원소 각각을 리스트로 감싼 다음 다시 합친다.
    return result

In [6]:
get_binary_split(pd_data,'income')

[['high'],
 ['medium'],
 ['low'],
 ['high', 'medium'],
 ['high', 'low'],
 ['medium', 'low']]

## 모든 이진분류의 경우의 Gini index를 구하는 함수 만들기
- 위에서 완성한 두 함수를 사용하여 만들어주세요!
- 해당 결과는 아래와 같이 나와야 합니다.

In [7]:
# attribute : 데이터를 split하는 컬럼 , label : class
def get_attribute_gini_index(df, attribute, label):
    
    result = {}
    binary_split = get_binary_split(df, attribute)
    n = len(df)
    
    # binary_split 원소 각각에 대한 지니계수를 넣을 리스트 생성
    lst = []
    i=0
    
    for a in binary_split :
        
        # 첫 번째 원소에 해당하는 데이터프레임과, 그 원소를 뺀 데이터프레임 각각 생성
        data1 = df[df[attribute].isin(a)]; data2 = df[~df[attribute].isin(a)]
        
        # 첫 번째 원소에 대한 지니계수 구하기
        gini = (len(data1)/n)*(get_gini(data1,label)) + (len(data2)/n)*(get_gini(data2,label))
        # 리스트에 삽입
        lst.append(gini)
        
        # 딕셔너리에 삽입
        result[tuple(a)] = lst[i]
        i += 1
    
    return result

In [8]:
get_attribute_gini_index(pd_data, 'income', 'class_buys_computer')

{('high',): 0.4428571428571429,
 ('medium',): 0.4583333333333333,
 ('low',): 0.45,
 ('high', 'medium'): 0.45,
 ('high', 'low'): 0.4583333333333333,
 ('medium', 'low'): 0.4428571428571429}

- 여기서 가장 작은 Gini index값을 가지는 class를 확인합니다.

In [9]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())

(('high',), 0.4428571428571429)

In [10]:
min(get_attribute_gini_index(pd_data, 'income', 'class_buys_computer').items())[0]

('high',)

## 분류를 하는 데 가장 중요한 변수를 선정하고, 해당 변수의 Gini index를 제시해주세요.
- 모든 변수에 대한 Gini index(최소)를 출력해주세요.
- 해당 결과는 아래와 같이 나와야 합니다.

In [16]:
# 변수명 중 마지막에 위치한 label 컬럼 얻기
label = pd_data.columns[-1]
# label 변수를 제외한 변수명 얻기
features = list(pd_data.columns[:-1])

# 각 변수를 대상으로 반복문 수행(해당 변수 중 가장 낮은 gini 계수와 변수 출력)
for feature in features:
    print("Minimum Gini Index of ", feature, ": ", 
         round(min(get_attribute_gini_index(pd_data, feature, label).items())[1],4))

Minimum Gini Index of  age :  0.3571
Minimum Gini Index of  income :  0.4429
Minimum Gini Index of  student :  0.3673
Minimum Gini Index of  credit_rating :  0.4286


gini index가 가장 작게 나온 'age'를 가장 중요한 변수로 선정합니다.

이어서 해당 변수의 이진 분류된 각 class에 대해 Gini index도 계산합니다.

In [17]:
get_attribute_gini_index(pd_data, 'age', 'class_buys_computer')

{('youth',): 0.3936507936507937,
 ('middle_aged',): 0.35714285714285715,
 ('senior',): 0.45714285714285713,
 ('youth', 'middle_aged'): 0.45714285714285713,
 ('youth', 'senior'): 0.35714285714285715,
 ('middle_aged', 'senior'): 0.3936507936507937}

'age' 변수에서 gini index가 가장 작게 나온 'middle_aged' class를 선정합니다.

## Entropy 를 구하는 함수 만들기

<img src = https://miro.medium.com/max/1122/0*DkWdyGidNSfdT1Nu.png width = "350">

In [91]:
from math import log2

def getEntropy(df, feature) :
    
    data = df[feature]
    
    entropy = 0
    for a in data.unique():
        p = sum(data==a)/len(data)
        entropy -= p*log2(p)
    
    return entropy

In [51]:
getEntropy(pd_data, "class_buys_computer")

0.9402859586706311

### Entropy

In [184]:
# 가장 중요한 변수로 선정된 목표변수를 제외한 다른 변수들에 대해
# 각 칼럼별로 엔트로피를 구해주는 함수를 작성해주세요.

def getGainA(df, feature) :
        
    result = {}
    
    # 목표변수를 제외한 컬럼 가져오기
    columns = list(pd_data.columns[:-1])
    for column in columns:
        
        # 컬럼 1개와 목표변수를 합친(2개의 열로 구성된) 데이터프레임 생성
        data1 = df[[column, feature]]
        
        # 엔트로피의 초기값을 0으로
        entropy = 0
        
        for a in data1.iloc[:,0].unique():
            # 첫 번째 컬럼의 각 원소별로 다시 데이터프레임 생성
            data2 = data1[data1.iloc[:,0] == a]
            # 컬럼별 엔트로피 구하기
            entropy += (len(data2)/len(data1))*getEntropy(data2, feature)
        
        result[column] = entropy
        
    return(result)

In [185]:
# Entropy
getGainA(pd_data, "class_buys_computer")

{'age': 0.6935361388961918,
 'income': 0.9110633930116763,
 'student': 0.7884504573082896,
 'credit_rating': 0.8921589282623617}

### Information Gain

In [181]:
def getGainA(df, feature) :
        
    result = {}
    
    columns = list(pd_data.columns[:-1])
    for column in columns:
        data1 = df[[column, feature]]
        
        entropy = 0
        lst = []
        for a in data1.iloc[:,0].unique():
            data2 = data1[data1.iloc[:,0] == a]
            entropy += (len(data2)/len(data1))*getEntropy(data2, feature)
            # 위의 코드와 다른 점은 루트노드의 엔트로피에서 컬럼별 엔트로피를 뺀 코드를 추가했습니다.
            ig = getEntropy(df, feature) - entropy
        result[column] = ig
        
    return(result)

In [182]:
# information gain
getGainA(pd_data, "class_buys_computer")

{'age': 0.24674981977443933,
 'income': 0.02922256565895487,
 'student': 0.15183550136234159,
 'credit_rating': 0.04812703040826949}